# Трекинг

In [ ]:
# Подключение к гугл-диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Клонируем репозиторий и устанавливаем зависимости
!git clone --recurse-submodules https://github.com/apchooo/yolov8_tracking.git  # clone recursively
!pip install -r /content/yolov8_tracking/requirements.txt  # install dependencies

fatal: destination path 'yolov8_tracking' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Импортируем библиотеки
import os
import pandas as pd
import cv2

In [ ]:
# Создаем папку weights в папке репозитория (требуется при запуске трекера)
!mkdir /content/yolov8_tracking/weights

In [ ]:
# Проверяем наличие папки weights в папке репозитория
!ls /content/yolov8_tracking/

assets	      Dockerfile  LICENSE    requirements.txt  track.py  weights
CITATION.cff  evolve.py   README.md  trackers	       val.py	 yolov8


In [ ]:
# Запускаем трекер
!python /content/yolov8_tracking/track.py --yolo-weights /content/drive/MyDrive/Hardhats_vests/yolov8s.pt --tracking-method strongsort --source /content/drive/MyDrive/Hardhats_vests/test/1.mp4 --save-txt --save-conf --save-trajectories --save-vid


track: yolo_weights=[PosixPath('/content/drive/MyDrive/Hardhats_vests/yolov8s.pt')], reid_weights=/content/yolov8_tracking/weights/osnet_x0_25_msmt17.pt, tracking_method=strongsort, tracking_config=yolov8_tracking/trackers/strongsort/configs/strongsort.yaml, source=/content/drive/MyDrive/Hardhats_vests/test/1.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=False, save_txt=True, save_conf=True, save_crop=False, save_trajectories=True, save_vid=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov8_tracking/runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, vid_stride=1, retina_masks=False
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
Successfully loaded pretrained weights from "/content/yolov8_tracking/weights/osn

# Постобработка

In [ ]:
# Трекер сохранил результаты в yolov8_tracking/runs/track/exp5
# Находим нужный файл
data_file = '/content/yolov8_tracking/runs/track/exp5/tracks/1.txt'
filepath = '/content/drive/MyDrive/Hardhats_vests/tracker_results/1/tracks/1.txt'

In [ ]:
# Загружаем данные из файла в Pandas DataFrame
data = pd.read_csv(filepath, header=None, sep=' ', usecols=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])
data.columns = ["frame", "id", "class", "confidence", "bb_left", "bb_top", "bb_right", "bb_bottom", "cntrx", "cntry"]

In [ ]:
data.head()

,frame,id,class,confidence,bb_left,bb_top,bb_right,bb_bottom,cntrx,cntry
0,83,1,2,0.904746,268,207,327,289,297,248
1,83,2,0,0.850198,257,157,335,386,296,271
2,84,1,2,0.904765,267,206,328,290,297,248
3,84,2,0,0.850209,258,156,336,386,297,271
4,85,1,2,0.859295,273,202,337,290,305,246


In [ ]:
# Сортируем по значениям кадров и классов (по возрастанию)
data.sort_values(['frame','class'], ascending=[True, True], inplace=True)
data.head(10)

,frame,id,class,confidence,bb_left,bb_top,bb_right,bb_bottom,cntrx,cntry
1,83,2,0,0.850198,257,157,335,386,296,271
0,83,1,2,0.904746,268,207,327,289,297,248
3,84,2,0,0.850209,258,156,336,386,297,271
2,84,1,2,0.904765,267,206,328,290,297,248
5,85,2,0,0.855229,264,151,343,377,303,264
6,85,3,1,0.746842,297,153,341,182,319,167
4,85,1,2,0.859295,273,202,337,290,305,246
8,86,2,0,0.855378,265,151,344,376,304,263
9,86,3,1,0.748865,296,152,342,183,319,167
7,86,1,2,0.859451,272,201,338,291,305,246


In [ ]:
# Создаем мультииндекс
data.set_index(['frame', 'class'], inplace=True)
data.head(10)

id  confidence  bb_left  bb_top  bb_right  bb_bottom  cntrx  \
frame class                                                                
83    0       2    0.850198      257     157       335        386    296   
      2       1    0.904746      268     207       327        289    297   
84    0       2    0.850209      258     156       336        386    297   
      2       1    0.904765      267     206       328        290    297   
85    0       2    0.855229      264     151       343        377    303   
      1       3    0.746842      297     153       341        182    319   
      2       1    0.859295      273     202       337        290    305   
86    0       2    0.855378      265     151       344        376    304   
      1       3    0.748865      296     152       342        183    319   
      2       1    0.859451      272     201       338        291    305   

             cntry  
frame class         
83    0        271  
      2        248  
84    0        271  
      2        248  
85    0        264  
      1        167  
      2        246  
86    0        263  
      1        167  
      2        246

In [ ]:
# Задаем функцию who_came_left, принимает датафрейм, возвращает id людей, которые вошли и вышли.
def who_came_left(data):
    frame_top = data['bb_top'].min() # кадры иногда имеют черную рамку, ищем границы картинки
    frame_bottom = data['bb_bottom'].max()
    ppl_who_came = []
    ppl_who_left = []
    for idx, df in data.groupby(['class', 'id']):
        if idx[0] == 0: # only for human class
            print(idx)
            cntry_min = df['cntry'].min() # человек иногда заходит и сразу выходит
            cntry_max = df['cntry'].max() # поэтому учитываем не только его начальную и конечную координаты,
            cntry_start = df['cntry'].iat[0] # но и его минимальную и максимальную координаты.
            cntry_end = df['cntry'].iat[-1]
            print(f'cntry_min = {cntry_min}, cntry_max = {cntry_max}')
            print(f'cntry_start = {cntry_start}, cntry_end = {cntry_end}')
            moved = (cntry_max - cntry_min)/(frame_bottom - frame_top)
            print(f'Human {idx[1]} moved {int(100*moved)}% of the frame (vertically)')
            if moved >= 0.25:        # если центр бб человека переместился более 25% кадра по вертикали, считаем прошедшим
                if (0.8*cntry_min < cntry_start < 1.2*cntry_min and # если человек начал путь примерно наверху кадра
                    0.8*cntry_max < cntry_end < 1.2*cntry_max): # если человек закончил путь примерно внизу кадра
                    ppl_who_came.append(idx[1])                 # то мы засчитываем его как вошедшего
                    print(f'Human {idx[1]} came in.')
                elif (0.8*cntry_min < cntry_end < 1.2*cntry_min and # если человек закончил путь примерно наверху кадра
                    0.8*cntry_max < cntry_start < 1.2*cntry_max): # если человек начал путь примерно внизу кадра
                    ppl_who_left.append(idx[1])                 # то мы засчитываем его как вышедшего
                    print(f'Human {idx[1]} left.')
                elif (0.8*cntry_min < cntry_start < 1.2*cntry_min and # если человек начал путь примерно наверху кадра
                    0.8*cntry_min < cntry_end < 1.2*cntry_min): # если человек закончил путь примерно наверху кадра
                    ppl_who_came.append(idx[1])                 # то мы считаем, что он вошел и сразу вышел
                    ppl_who_left.append(idx[1])
                    print(f'Human {idx[1]} came and left.')
                elif (0.8*cntry_max < cntry_start < 1.2*cntry_max and # если человек начал путь примерно внизу кадра
                    0.8*cntry_max < cntry_end < 1.2*cntry_max): # если человек закончил путь примерно внизу кадра
                    ppl_who_came.append(idx[1])                 # то мы считаем, что он вышел и сразу вернулся
                    ppl_who_left.append(idx[1])
                    print(f'Human {idx[1]} left and came back.')
                else:
                    print(f'Can not define what human {idx[1]} did.')
    return ppl_who_came, ppl_who_left

In [ ]:
# Применим функцию who_came_left и получим списки id прошедших людей.
ppl_who_came, ppl_who_left = who_came_left(data)
print(ppl_who_came, ppl_who_left)
print(f'{len(ppl_who_came)} people came in')
print(f'{len(ppl_who_left)} people left')

(0, 2)
cntry_min = 249, cntry_max = 394
cntry_start = 271, cntry_end = 394
Human 2 moved 40% of the frame (vertically)
Human 2 came in.
(0, 6)
cntry_min = 242, cntry_max = 425
cntry_start = 248, cntry_end = 425
Human 6 moved 50% of the frame (vertically)
Human 6 came in.
(0, 7)
cntry_min = 266, cntry_max = 398
cntry_start = 266, cntry_end = 398
Human 7 moved 36% of the frame (vertically)
Human 7 came in.
(0, 10)
cntry_min = 384, cntry_max = 396
cntry_start = 391, cntry_end = 384
Human 10 moved 3% of the frame (vertically)
[2, 6, 7] []
3 people came in
0 people left


In [ ]:
# Зададим функцию get_who_wears_what,
# на вход - часть датафрейма с инфо по одному кадру
# на выход - список кортежей, где каждый кортеж это:
# (id человека, True/False на наличие каски, True/False на наличие жилета)
def get_who_wears_what(current_frame):
    try:
        humans_frame = current_frame.loc[0]
        humans = True
        if isinstance(humans_frame, pd.Series):
            humans_frame = humans_frame.to_frame().transpose()
    except:
        # print(f"There are no humans in frame {frame}.")
        humans = False
    try:
        hardhats_frame = current_frame.loc[1]
        hardhats = True
        if isinstance(hardhats_frame, pd.Series):
            hardhats_frame = hardhats_frame.to_frame().transpose()
    except:
        # print(f"There are no hardhats in frame {frame}.")
        hardhats = False
    try:
        vests_frame = current_frame.loc[2]
        vests = True
        if isinstance(vests_frame, pd.Series):
            vests_frame = vests_frame.to_frame().transpose()
    except:
        # print(f"There are no vests in frame {frame}.")
        vests = False
    frame_info = []
    if humans:
        for index, row in humans_frame.iterrows():
            current_human = [int(row['id']), False, False]
            if hardhats:
                for idx, rw in hardhats_frame.iterrows():
                    if row['bb_left'] < rw['cntrx'] < row['bb_right'] and row['bb_top'] < rw['cntry'] < row['bb_bottom']:
                        # print(f"human {row['id']} has a hat")
                        current_human[1] = True
            if vests:            
                for idx, rw in vests_frame.iterrows():
                    if row['bb_left'] < rw['cntrx'] < row['bb_right'] and row['bb_top'] < rw['cntry'] < row['bb_bottom']:
                        # print(f"human {row['id']} has a vest")
                        current_human[2] = True
            current_human = tuple(current_human)
            # print(current_human)
            frame_info.append(current_human)
    # print(frame_info)
    return frame_info

In [ ]:
# Используем созданную функцию get_who_wears_what на данных
# Получаем словарь info:
# ключи - номера кадров,
# значения - списки кортежей, где каждый кортеж это:
# (id человека, True/False на наличие каски, True/False на наличие жилета)
info = {}
for frame, new_df in data.groupby(level=0):
    current_frame = new_df.droplevel(0)
    frame_info = get_who_wears_what(current_frame)
    info[frame] = frame_info
print(info)

{83: [(2, False, True)], 84: [(2, False, True)], 85: [(2, True, True)], 86: [(2, True, True)], 87: [(2, True, True)], 88: [(2, True, True)], 89: [(2, True, True)], 90: [(2, True, True)], 91: [(2, True, True)], 92: [(2, True, True)], 93: [(2, True, True)], 94: [(2, True, True)], 95: [(2, True, True), (6, True, True)], 96: [(2, True, True), (6, True, True)], 97: [(2, True, True), (6, True, True)], 98: [(2, True, True), (6, True, True)], 99: [(2, True, True), (6, True, True)], 100: [(2, True, True)], 101: [(2, True, True)], 102: [(2, True, True)], 103: [(2, True, True)], 104: [(2, True, True)], 105: [(2, True, True)], 106: [(2, True, True)], 107: [(2, True, True), (7, False, True)], 108: [(2, True, True), (7, False, True)], 109: [(2, True, True), (7, True, True)], 110: [(2, True, True), (7, True, True)], 111: [(2, True, True), (6, True, True), (7, True, True)], 112: [(2, True, True), (6, True, True)], 113: [(2, True, True), (6, True, True)], 114: [(2, True, True), (6, True, True)], 115: [

In [ ]:
filepath = '/content/drive/MyDrive/Hardhats_vests/tracker_results/1/1.mp4'
video = cv2.VideoCapture(filepath)
fps = video.get(cv2.CAP_PROP_FPS)
fps

12.025

In [ ]:
# Теперь, когда у нас есть словарь info, мы можем оценить в каком количестве кадров у человека детектится каска и жилет
def get_good_bad_human(id, info, fps):
    human_frames = []
    hardhat_frames = []
    vest_frames = []
    for key, value in info.items():
        for human in value:
            if human[0] == id:
                human_frames.append(key)
                if human[1] == True:
                    hardhat_frames.append(key)
                if human[2] == True:
                    vest_frames.append(key)
    start_frame = min(human_frames)
    end_frame = max(human_frames)
    start_time = int(start_frame//fps)
    end_time = int(end_frame//fps)
    human_frames_n = len(human_frames)
    hardhat_frames_n = len(hardhat_frames)
    vest_frames_n = len(vest_frames)
    violations_dict = {0 : 'all good!', 1 : 'no hat, no vest', 2 : 'no vest', 3 : 'no hat'}
    print(f'Human {id} is detected in {human_frames_n} frames, we detect that he has a hat in {hardhat_frames_n} frames and vest in {vest_frames_n} frames.')
    if hardhat_frames_n/human_frames_n >= 0.7 and vest_frames_n/human_frames_n >= 0.7:
        violation_key = 0
        print(f"Human {id} is a good human. :-)")
    elif hardhat_frames_n/human_frames_n >= 0.7:
        violation_key = 2
        print(f'Human with id {id} is crossing with {violations_dict[violation_key]} in interval {start_time//60} min {start_time%60} sec - {end_time//60} min {end_time%60} sec.')
    elif vest_frames_n/human_frames_n >= 0.7:
        violation_key = 3
        print(f'Human with id {id} is crossing with {violations_dict[violation_key]} in interval {start_time//60} min {start_time%60} sec - {end_time//60} min {end_time%60} sec.')
    else:
        violation_key = 1
        print(f'Human with id {id} is crossing with {violations_dict[violation_key]} in interval {start_time//60} min {start_time%60} sec - {end_time//60} min {end_time%60} sec.')
    return id, violation_key, start_time, end_time


In [ ]:
# Применим функцию get_good_bad_human к полученным ранее спискам вошедших и вышедших
for id in ppl_who_came:
    id, violation_key, start_time, end_time = get_good_bad_human(id, info, fps)

Human 2 is detected in 47 frames, we detect that he has a hat in 43 frames and vest in 47 frames.
Human 2 is a good human. :-)
Human 6 is detected in 54 frames, we detect that he has a hat in 53 frames and vest in 52 frames.
Human 6 is a good human. :-)
Human 7 is detected in 42 frames, we detect that he has a hat in 38 frames and vest in 38 frames.
Human 7 is a good human. :-)


In [ ]:
for id in ppl_who_left:
    id, violation_key, start_time, end_time = get_good_bad_human(id, info, fps)

# Проверка работы алгоритма на данных с других видео

Здесь берем данные с гугл-диска, которые уже были получены на локальном GPU

 (чтобы здесь повторно не пересчитывать то же самое)

In [ ]:
directory = '/content/drive/MyDrive/Hardhats_vests/tracker_results'
# /content/drive/MyDrive/Hardhats_vests/tracker_results/1/tracks/1.txt

In [ ]:
for file_number in range(43):
  filepath = directory + '/' + str(file_number) + '/tracks/' + str(file_number) + '.txt'
  print(filepath)

In [ ]:
def get_results(filepath):
    # Загружаем данные из файла в Pandas DataFrame
    data = pd.read_csv(filepath, header=None, sep=' ', usecols=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ])
    data.columns = ["frame", "id", "class", "confidence", "bb_left", "bb_top", "bb_right", "bb_bottom", "cntrx", "cntry"]
    # Сортируем по значениям кадров и классов (по возрастанию)
    data.sort_values(['frame','class'], ascending=[True, True], inplace=True)
    # Создаем мультииндекс
    data.set_index(['frame', 'class'], inplace=True)

    # Применим функцию who_came_left и получим списки id прошедших людей.
    ppl_who_came, ppl_who_left = who_came_left(data)
    ppl_who_crossed = ppl_who_came + ppl_who_left
    print(ppl_who_came, ppl_who_left)
    came = len(ppl_who_came)
    left = len(ppl_who_left)
    print(f'{came} people came in')
    print(f'{left} people left')

    # Используем созданную функцию get_who_wears_what на данных
    # Получаем словарь info:
    # ключи - номера кадров,
    # значения - списки кортежей, где каждый кортеж это:
    # (id человека, True/False на наличие каски, True/False на наличие жилета)
    info = {}
    for frame, new_df in data.groupby(level=0):
        current_frame = new_df.droplevel(0)
        frame_info = get_who_wears_what(current_frame)
        info[frame] = frame_info
    print(info)

    # Применим функцию get_good_bad_human к полученным ранее спискам вошедших и вышедших
    violations = {}
    for id in ppl_who_crossed:
        id, violation_key, start_time, end_time = get_good_bad_human(id, info, fps)
        if violation_key != 0:
            if violation_key in violations:
                violations[violation_key].append((start_time, end_time))
            else:
                violations[violation_key] = [(start_time, end_time)]
    return came, left, violations

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/1/tracks/1.txt')
print(came, left, violations)

(0, 2)
cntry_min = 249, cntry_max = 394
cntry_start = 271, cntry_end = 394
Human 2 moved 40% of the frame (vertically)
Human 2 came in.
(0, 6)
cntry_min = 242, cntry_max = 425
cntry_start = 248, cntry_end = 425
Human 6 moved 50% of the frame (vertically)
Human 6 came in.
(0, 7)
cntry_min = 266, cntry_max = 398
cntry_start = 266, cntry_end = 398
Human 7 moved 36% of the frame (vertically)
Human 7 came in.
(0, 10)
cntry_min = 384, cntry_max = 396
cntry_start = 391, cntry_end = 384
Human 10 moved 3% of the frame (vertically)
[2, 6, 7] []
3 people came in
0 people left
{83: [(2, False, True)], 84: [(2, False, True)], 85: [(2, True, True)], 86: [(2, True, True)], 87: [(2, True, True)], 88: [(2, True, True)], 89: [(2, True, True)], 90: [(2, True, True)], 91: [(2, True, True)], 92: [(2, True, True)], 93: [(2, True, True)], 94: [(2, True, True)], 95: [(2, True, True), (6, True, True)], 96: [(2, True, True), (6, True, True)], 97: [(2, True, True), (6, True, True)], 98: [(2, True, True), (6, Tru

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/2/tracks/2.txt')
print(came, left, violations)

(0, 1)
cntry_min = 276, cntry_max = 409
cntry_start = 278, cntry_end = 409
Human 1 moved 38% of the frame (vertically)
Human 1 came in.
(0, 5)
cntry_min = 383, cntry_max = 398
cntry_start = 398, cntry_end = 383
Human 5 moved 4% of the frame (vertically)
(0, 9)
cntry_min = 388, cntry_max = 404
cntry_start = 390, cntry_end = 403
Human 9 moved 4% of the frame (vertically)
(0, 11)
cntry_min = 388, cntry_max = 416
cntry_start = 399, cntry_end = 407
Human 11 moved 8% of the frame (vertically)
(0, 16)
cntry_min = 291, cntry_max = 414
cntry_start = 291, cntry_end = 414
Human 16 moved 35% of the frame (vertically)
Human 16 came in.
(0, 20)
cntry_min = 294, cntry_max = 417
cntry_start = 294, cntry_end = 417
Human 20 moved 35% of the frame (vertically)
Human 20 came in.
(0, 22)
cntry_min = 391, cntry_max = 417
cntry_start = 398, cntry_end = 396
Human 22 moved 7% of the frame (vertically)
(0, 27)
cntry_min = 422, cntry_max = 427
cntry_start = 422, cntry_end = 427
Human 27 moved 1% of the frame (ve

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/3/tracks/3.txt')
print(came, left, violations)

(0, 1)
cntry_min = 296, cntry_max = 332
cntry_start = 296, cntry_end = 325
Human 1 moved 10% of the frame (vertically)
(0, 4)
cntry_min = 275, cntry_max = 308
cntry_start = 275, cntry_end = 284
Human 4 moved 9% of the frame (vertically)
[] []
0 people came in
0 people left
{46: [(1, False, False)], 47: [(1, False, False)], 48: [(1, False, False)], 49: [(1, False, False)], 50: [(1, False, False)], 51: [(1, False, False)], 52: [(1, False, False)], 53: [(1, False, False)], 54: [(1, False, False)], 55: [(1, False, False)], 56: [(1, False, False)], 57: [(1, False, False)], 58: [(1, False, False)], 59: [(1, False, False)], 60: [(1, False, False)], 61: [(1, False, False)], 62: [(1, False, False)], 63: [(1, False, False)], 64: [(1, False, False)], 65: [(1, False, False)], 66: [(1, False, False)], 67: [(1, False, False)], 68: [(1, False, False), (4, True, True)], 69: [(1, False, False), (4, True, True)], 70: [(1, False, False), (4, True, True)], 71: [(1, False, False), (4, True, True)], 72: [(1

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/4/tracks/4.txt')
print(came, left, violations)

(0, 1)
cntry_min = 575, cntry_max = 827
cntry_start = 575, cntry_end = 827
Human 1 moved 35% of the frame (vertically)
Human 1 came in.
(0, 2)
cntry_min = 853, cntry_max = 865
cntry_start = 853, cntry_end = 865
Human 2 moved 1% of the frame (vertically)
(0, 6)
cntry_min = 562, cntry_max = 832
cntry_start = 562, cntry_end = 832
Human 6 moved 38% of the frame (vertically)
Human 6 came in.
(0, 7)
cntry_min = 768, cntry_max = 786
cntry_start = 784, cntry_end = 768
Human 7 moved 2% of the frame (vertically)
(0, 9)
cntry_min = 573, cntry_max = 590
cntry_start = 573, cntry_end = 576
Human 9 moved 2% of the frame (vertically)
(0, 12)
cntry_min = 757, cntry_max = 787
cntry_start = 787, cntry_end = 757
Human 12 moved 4% of the frame (vertically)
(0, 15)
cntry_min = 583, cntry_max = 796
cntry_start = 588, cntry_end = 796
Human 15 moved 30% of the frame (vertically)
Human 15 came in.
(0, 18)
cntry_min = 562, cntry_max = 830
cntry_start = 564, cntry_end = 830
Human 18 moved 37% of the frame (vertic

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/5/tracks/5.txt')
print(came, left, violations)

(0, 1)
cntry_min = 290, cntry_max = 338
cntry_start = 292, cntry_end = 290
Human 1 moved 13% of the frame (vertically)
(0, 3)
cntry_min = 337, cntry_max = 429
cntry_start = 396, cntry_end = 392
Human 3 moved 26% of the frame (vertically)
Human 3 came in.
(0, 10)
cntry_min = 285, cntry_max = 301
cntry_start = 292, cntry_end = 293
Human 10 moved 4% of the frame (vertically)
[3] []
1 people came in
0 people left
{51: [(1, False, True)], 52: [(1, False, True)], 53: [(1, False, True)], 54: [(1, False, True)], 55: [(1, False, True)], 56: [(1, False, True)], 57: [(1, False, True), (3, False, False)], 58: [(1, False, True), (3, False, False)], 59: [(1, False, True), (3, False, True)], 60: [(1, False, True), (3, False, True)], 61: [(1, False, True), (3, False, True)], 62: [(1, False, True), (3, False, True)], 63: [(1, False, True), (3, False, True)], 64: [(1, False, False), (3, False, True)], 65: [(1, False, False), (3, False, True)], 66: [(1, False, False), (3, False, False)], 67: [(1, False, 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/6/tracks/6.txt')
print(came, left, violations)

(0, 1)
cntry_min = 215, cntry_max = 451
cntry_start = 399, cntry_end = 223
Human 1 moved 37% of the frame (vertically)
Human 1 left.
(0, 3)
cntry_min = 479, cntry_max = 498
cntry_start = 498, cntry_end = 479
Human 3 moved 3% of the frame (vertically)
(0, 4)
cntry_min = 248, cntry_max = 426
cntry_start = 424, cntry_end = 248
Human 4 moved 28% of the frame (vertically)
Human 4 left.
(0, 5)
cntry_min = 300, cntry_max = 344
cntry_start = 344, cntry_end = 300
Human 5 moved 6% of the frame (vertically)
(0, 6)
cntry_min = 267, cntry_max = 295
cntry_start = 295, cntry_end = 267
Human 6 moved 4% of the frame (vertically)
(0, 9)
cntry_min = 234, cntry_max = 397
cntry_start = 397, cntry_end = 234
Human 9 moved 25% of the frame (vertically)
Human 9 left.
(0, 15)
cntry_min = 224, cntry_max = 424
cntry_start = 224, cntry_end = 424
Human 15 moved 31% of the frame (vertically)
Human 15 came in.
(0, 20)
cntry_min = 441, cntry_max = 447
cntry_start = 447, cntry_end = 442
Human 20 moved 0% of the frame (

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/7/tracks/7.txt')
print(came, left, violations)

(0, 2)
cntry_min = 177, cntry_max = 346
cntry_start = 346, cntry_end = 194
Human 2 moved 26% of the frame (vertically)
Human 2 left.
(0, 6)
cntry_min = 187, cntry_max = 431
cntry_start = 374, cntry_end = 187
Human 6 moved 38% of the frame (vertically)
Human 6 left.
(0, 8)
cntry_min = 365, cntry_max = 414
cntry_start = 414, cntry_end = 365
Human 8 moved 7% of the frame (vertically)
(0, 11)
cntry_min = 451, cntry_max = 462
cntry_start = 462, cntry_end = 451
Human 11 moved 1% of the frame (vertically)
(0, 12)
cntry_min = 224, cntry_max = 503
cntry_start = 458, cntry_end = 234
Human 12 moved 43% of the frame (vertically)
Human 12 left.
(0, 15)
cntry_min = 418, cntry_max = 421
cntry_start = 421, cntry_end = 418
Human 15 moved 0% of the frame (vertically)
(0, 19)
cntry_min = 224, cntry_max = 450
cntry_start = 450, cntry_end = 224
Human 19 moved 35% of the frame (vertically)
Human 19 left.
[] [2, 6, 12, 19]
0 people came in
4 people left
{42: [], 43: [], 44: [(2, False, True)], 45: [(2, False

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/8/tracks/8.txt')
print(came, left, violations)

(0, 1)
cntry_min = 301, cntry_max = 403
cntry_start = 403, cntry_end = 301
Human 1 moved 27% of the frame (vertically)
Human 1 left.
(0, 2)
cntry_min = 369, cntry_max = 376
cntry_start = 376, cntry_end = 369
Human 2 moved 1% of the frame (vertically)
(0, 3)
cntry_min = 380, cntry_max = 405
cntry_start = 392, cntry_end = 392
Human 3 moved 6% of the frame (vertically)
(0, 4)
cntry_min = 268, cntry_max = 355
cntry_start = 355, cntry_end = 268
Human 4 moved 23% of the frame (vertically)
(0, 7)
cntry_min = 266, cntry_max = 387
cntry_start = 381, cntry_end = 266
Human 7 moved 33% of the frame (vertically)
Human 7 left.
(0, 8)
cntry_min = 396, cntry_max = 399
cntry_start = 398, cntry_end = 396
Human 8 moved 0% of the frame (vertically)
(0, 10)
cntry_min = 275, cntry_max = 377
cntry_start = 372, cntry_end = 276
Human 10 moved 27% of the frame (vertically)
Human 10 left.
(0, 11)
cntry_min = 417, cntry_max = 417
cntry_start = 417, cntry_end = 417
Human 11 moved 0% of the frame (vertically)
(0, 1

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/9/tracks/9.txt')
print(came, left, violations)

(0, 2)
cntry_min = 271, cntry_max = 393
cntry_start = 393, cntry_end = 279
Human 2 moved 35% of the frame (vertically)
Human 2 left.
(0, 3)
cntry_min = 340, cntry_max = 385
cntry_start = 385, cntry_end = 377
Human 3 moved 13% of the frame (vertically)
(0, 11)
cntry_min = 388, cntry_max = 417
cntry_start = 388, cntry_end = 417
Human 11 moved 8% of the frame (vertically)
(0, 16)
cntry_min = 286, cntry_max = 404
cntry_start = 286, cntry_end = 404
Human 16 moved 34% of the frame (vertically)
Human 16 came in.
(0, 21)
cntry_min = 370, cntry_max = 376
cntry_start = 371, cntry_end = 376
Human 21 moved 1% of the frame (vertically)
[16] [2]
1 people came in
1 people left
{94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: [], 101: [], 102: [(2, False, True)], 103: [(2, False, True)], 104: [(2, False, True)], 105: [(2, False, True)], 106: [(2, False, True)], 107: [(2, False, True)], 108: [(2, False, True)], 109: [(2, False, True)], 110: [(2, False, True)], 111: [(2, False, True)], 112: [(2, Fa

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/10/tracks/10.txt')
print(came, left, violations)

(0, 2)
cntry_min = 271, cntry_max = 384
cntry_start = 367, cntry_end = 279
Human 2 moved 31% of the frame (vertically)
Human 2 left.
(0, 6)
cntry_min = 400, cntry_max = 411
cntry_start = 409, cntry_end = 400
Human 6 moved 3% of the frame (vertically)
(0, 8)
cntry_min = 346, cntry_max = 396
cntry_start = 396, cntry_end = 346
Human 8 moved 14% of the frame (vertically)
[] [2]
0 people came in
1 people left
{49: [(2, False, True)], 50: [(2, False, True)], 51: [(2, False, True)], 52: [(2, False, True)], 53: [(2, False, True)], 54: [(2, False, True)], 55: [(2, False, True)], 56: [(2, False, True)], 57: [(2, False, True)], 58: [(2, False, True)], 59: [(2, False, True)], 60: [(2, False, True)], 61: [(2, False, True)], 62: [(2, False, True)], 63: [(2, False, True)], 64: [(2, False, True)], 65: [(2, False, True)], 66: [(2, False, True)], 67: [(2, False, True)], 68: [(2, False, True)], 69: [(2, False, True)], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [(2, False, True)], 78: [(2

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/11/tracks/11.txt')
print(came, left, violations)

(0, 1)
cntry_min = 271, cntry_max = 395
cntry_start = 395, cntry_end = 271
Human 1 moved 35% of the frame (vertically)
Human 1 left.
(0, 2)
cntry_min = 378, cntry_max = 407
cntry_start = 407, cntry_end = 392
Human 2 moved 8% of the frame (vertically)
(0, 3)
cntry_min = 372, cntry_max = 382
cntry_start = 380, cntry_end = 374
Human 3 moved 2% of the frame (vertically)
(0, 4)
cntry_min = 417, cntry_max = 418
cntry_start = 418, cntry_end = 417
Human 4 moved 0% of the frame (vertically)
(0, 5)
cntry_min = 413, cntry_max = 417
cntry_start = 416, cntry_end = 413
Human 5 moved 1% of the frame (vertically)
(0, 7)
cntry_min = 270, cntry_max = 394
cntry_start = 394, cntry_end = 270
Human 7 moved 35% of the frame (vertically)
Human 7 left.
(0, 9)
cntry_min = 282, cntry_max = 390
cntry_start = 390, cntry_end = 282
Human 9 moved 30% of the frame (vertically)
Human 9 left.
[] [1, 7, 9]
0 people came in
3 people left
{26: [(1, False, False)], 27: [(1, False, False)], 28: [(1, False, False)], 29: [(1, 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/12/tracks/12.txt')
print(came, left, violations)

(0, 2)
cntry_min = 378, cntry_max = 409
cntry_start = 379, cntry_end = 409
Human 2 moved 8% of the frame (vertically)
(0, 3)
cntry_min = 268, cntry_max = 382
cntry_start = 377, cntry_end = 268
Human 3 moved 31% of the frame (vertically)
Human 3 left.
(0, 4)
cntry_min = 391, cntry_max = 399
cntry_start = 399, cntry_end = 391
Human 4 moved 2% of the frame (vertically)
(0, 5)
cntry_min = 269, cntry_max = 381
cntry_start = 381, cntry_end = 269
Human 5 moved 30% of the frame (vertically)
Human 5 left.
(0, 6)
cntry_min = 274, cntry_max = 390
cntry_start = 376, cntry_end = 274
Human 6 moved 31% of the frame (vertically)
Human 6 left.
(0, 7)
cntry_min = 270, cntry_max = 362
cntry_start = 362, cntry_end = 270
Human 7 moved 25% of the frame (vertically)
Human 7 left.
[] [3, 5, 6, 7]
0 people came in
4 people left
{64: [], 65: [], 66: [(2, False, True)], 67: [(2, False, True)], 68: [(2, False, True)], 69: [(2, False, True)], 70: [(2, False, True)], 71: [(2, False, True)], 72: [(2, False, True)], 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/13/tracks/13.txt')
print(came, left, violations)

(0, 2)
cntry_min = 265, cntry_max = 369
cntry_start = 369, cntry_end = 265
Human 2 moved 28% of the frame (vertically)
Human 2 left.
(0, 6)
cntry_min = 243, cntry_max = 399
cntry_start = 385, cntry_end = 256
Human 6 moved 42% of the frame (vertically)
Human 6 left.
(0, 9)
cntry_min = 227, cntry_max = 240
cntry_start = 227, cntry_end = 240
Human 9 moved 3% of the frame (vertically)
(0, 12)
cntry_min = 287, cntry_max = 386
cntry_start = 386, cntry_end = 311
Human 12 moved 27% of the frame (vertically)
Human 12 left.
(0, 16)
cntry_min = 266, cntry_max = 394
cntry_start = 379, cntry_end = 394
Human 16 moved 35% of the frame (vertically)
Human 16 left and came back.
(0, 18)
cntry_min = 365, cntry_max = 367
cntry_start = 367, cntry_end = 365
Human 18 moved 0% of the frame (vertically)
(0, 21)
cntry_min = 277, cntry_max = 407
cntry_start = 277, cntry_end = 407
Human 21 moved 35% of the frame (vertically)
Human 21 came in.
[16, 21] [2, 6, 12, 16]
2 people came in
4 people left
{49: [], 50: [],

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/14/tracks/14.txt')
print(came, left, violations)

(0, 2)
cntry_min = 427, cntry_max = 572
cntry_start = 463, cntry_end = 572
Human 2 moved 23% of the frame (vertically)
(0, 8)
cntry_min = 218, cntry_max = 421
cntry_start = 404, cntry_end = 218
Human 8 moved 33% of the frame (vertically)
Human 8 left.
(0, 21)
cntry_min = 426, cntry_max = 430
cntry_start = 430, cntry_end = 426
Human 21 moved 0% of the frame (vertically)
(0, 29)
cntry_min = 169, cntry_max = 441
cntry_start = 426, cntry_end = 169
Human 29 moved 44% of the frame (vertically)
Human 29 left.
(0, 31)
cntry_min = 490, cntry_max = 500
cntry_start = 500, cntry_end = 490
Human 31 moved 1% of the frame (vertically)
(0, 36)
cntry_min = 376, cntry_max = 430
cntry_start = 430, cntry_end = 376
Human 36 moved 8% of the frame (vertically)
(0, 38)
cntry_min = 258, cntry_max = 505
cntry_start = 505, cntry_end = 260
Human 38 moved 40% of the frame (vertically)
Human 38 left.
(0, 41)
cntry_min = 234, cntry_max = 240
cntry_start = 238, cntry_end = 234
Human 41 moved 0% of the frame (vertical

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/15/tracks/15.txt')
print(came, left, violations)

(0, 1)
cntry_min = 294, cntry_max = 422
cntry_start = 294, cntry_end = 416
Human 1 moved 37% of the frame (vertically)
Human 1 came in.
(0, 2)
cntry_min = 281, cntry_max = 288
cntry_start = 284, cntry_end = 288
Human 2 moved 2% of the frame (vertically)
(0, 3)
cntry_min = 297, cntry_max = 302
cntry_start = 297, cntry_end = 298
Human 3 moved 1% of the frame (vertically)
(0, 4)
cntry_min = 296, cntry_max = 411
cntry_start = 296, cntry_end = 411
Human 4 moved 33% of the frame (vertically)
Human 4 came in.
[1, 4] []
2 people came in
0 people left
{3: [(1, False, False)], 4: [(1, False, False)], 5: [(1, False, False)], 6: [(1, False, False)], 7: [(1, False, False)], 8: [(1, False, False)], 9: [(1, False, False)], 10: [(1, False, False)], 11: [(1, False, False)], 12: [(1, False, False)], 13: [(1, False, False)], 14: [(1, False, False)], 15: [(1, False, False)], 16: [(1, False, False)], 17: [(1, False, False)], 18: [(1, False, False)], 19: [(1, False, False)], 20: [(1, False, False)], 21: [(1

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/16/tracks/16.txt')
print(came, left, violations)

(0, 2)
cntry_min = 257, cntry_max = 352
cntry_start = 330, cntry_end = 259
Human 2 moved 25% of the frame (vertically)
Human 2 left.
(0, 5)
cntry_min = 389, cntry_max = 395
cntry_start = 395, cntry_end = 389
Human 5 moved 1% of the frame (vertically)
(0, 7)
cntry_min = 379, cntry_max = 381
cntry_start = 379, cntry_end = 381
Human 7 moved 0% of the frame (vertically)
(0, 8)
cntry_min = 375, cntry_max = 377
cntry_start = 377, cntry_end = 375
Human 8 moved 0% of the frame (vertically)
(0, 11)
cntry_min = 269, cntry_max = 385
cntry_start = 385, cntry_end = 283
Human 11 moved 30% of the frame (vertically)
Human 11 left.
(0, 14)
cntry_min = 257, cntry_max = 393
cntry_start = 393, cntry_end = 273
Human 14 moved 35% of the frame (vertically)
Human 14 left.
(0, 16)
cntry_min = 408, cntry_max = 408
cntry_start = 408, cntry_end = 408
Human 16 moved 0% of the frame (vertically)
(0, 22)
cntry_min = 383, cntry_max = 385
cntry_start = 385, cntry_end = 383
Human 22 moved 0% of the frame (vertically)
(

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/17/tracks/17.txt')
print(came, left, violations)

(0, 1)
cntry_min = 269, cntry_max = 369
cntry_start = 369, cntry_end = 280
Human 1 moved 27% of the frame (vertically)
Human 1 left.
(0, 2)
cntry_min = 241, cntry_max = 391
cntry_start = 391, cntry_end = 267
Human 2 moved 41% of the frame (vertically)
Human 2 left.
(0, 4)
cntry_min = 362, cntry_max = 387
cntry_start = 387, cntry_end = 362
Human 4 moved 6% of the frame (vertically)
(0, 5)
cntry_min = 274, cntry_max = 372
cntry_start = 372, cntry_end = 289
Human 5 moved 26% of the frame (vertically)
Human 5 left.
(0, 10)
cntry_min = 396, cntry_max = 398
cntry_start = 398, cntry_end = 396
Human 10 moved 0% of the frame (vertically)
(0, 11)
cntry_min = 378, cntry_max = 398
cntry_start = 384, cntry_end = 395
Human 11 moved 5% of the frame (vertically)
(0, 12)
cntry_min = 246, cntry_max = 391
cntry_start = 391, cntry_end = 270
Human 12 moved 39% of the frame (vertically)
Human 12 left.
(0, 15)
cntry_min = 245, cntry_max = 378
cntry_start = 378, cntry_end = 245
Human 15 moved 36% of the frame

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/18/tracks/18.txt')
print(came, left, violations)

(0, 2)
cntry_min = 282, cntry_max = 403
cntry_start = 282, cntry_end = 403
Human 2 moved 33% of the frame (vertically)
Human 2 came in.
(0, 6)
cntry_min = 262, cntry_max = 407
cntry_start = 263, cntry_end = 407
Human 6 moved 40% of the frame (vertically)
Human 6 came in.
(0, 13)
cntry_min = 290, cntry_max = 405
cntry_start = 290, cntry_end = 405
Human 13 moved 31% of the frame (vertically)
Human 13 came in.
(0, 14)
cntry_min = 267, cntry_max = 398
cntry_start = 275, cntry_end = 398
Human 14 moved 36% of the frame (vertically)
Human 14 came in.
[2, 6, 13, 14] []
4 people came in
0 people left
{24: [], 25: [], 26: [], 27: [], 28: [(2, True, True)], 29: [(2, True, True)], 30: [(2, True, True)], 31: [(2, True, True)], 32: [(2, True, True)], 33: [(2, True, True)], 34: [(2, True, True)], 35: [(2, True, True)], 36: [(2, True, True)], 37: [(2, True, True)], 38: [(2, True, True)], 39: [(2, True, True)], 40: [(2, True, True)], 41: [(2, True, True)], 42: [(2, True, True)], 43: [(2, True, True)], 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/19/tracks/19.txt')
print(came, left, violations)

(0, 1)
cntry_min = 273, cntry_max = 387
cntry_start = 274, cntry_end = 387
Human 1 moved 33% of the frame (vertically)
Human 1 came in.
[1] []
1 people came in
0 people left
{24: [(1, False, True)], 25: [(1, False, True)], 26: [(1, False, True)], 27: [(1, False, True)], 28: [(1, False, True)], 29: [(1, False, True)], 30: [(1, False, True)], 31: [(1, False, True)], 32: [(1, False, True)], 33: [(1, False, True)], 34: [(1, False, True)], 35: [(1, False, True)], 36: [(1, False, True)], 37: [(1, False, True)], 38: [(1, False, True)], 39: [(1, False, True)], 40: [(1, False, True)], 41: [(1, False, True)], 42: [(1, False, True)], 43: [(1, False, True)], 44: [(1, False, True)], 45: [(1, False, True)], 46: [(1, False, True)], 47: [(1, False, True)], 48: [(1, False, True)], 49: [(1, False, True)], 50: [(1, False, True)], 51: [(1, False, True)], 52: [(1, False, True)], 53: [(1, False, True)], 54: [(1, False, True)], 55: [(1, False, True)], 56: [(1, False, True)], 57: [(1, False, True)], 58: [(1, 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/20/tracks/20.txt')
print(came, left, violations)

(0, 1)
cntry_min = 470, cntry_max = 732
cntry_start = 493, cntry_end = 732
Human 1 moved 28% of the frame (vertically)
Human 1 came in.
[1] []
1 people came in
0 people left
{61: [(1, False, False)], 62: [(1, False, False)], 63: [(1, False, False)], 64: [(1, False, False)], 65: [(1, False, False)], 66: [(1, False, False)], 67: [(1, False, False)], 68: [(1, False, False)], 69: [(1, False, False)], 70: [(1, False, False)], 71: [(1, False, False)], 72: [(1, False, False)], 73: [(1, False, False)], 74: [(1, False, False)], 75: [(1, False, False)], 76: [(1, False, False)], 77: [(1, False, False)], 78: [(1, False, False)], 79: [(1, False, False)], 80: [(1, False, False)], 81: [(1, False, False)], 82: [(1, False, False)], 83: [(1, False, False)], 84: [(1, False, False)], 85: [(1, False, False)], 86: [(1, False, False)], 87: [(1, False, False)], 88: [(1, False, False)], 89: [(1, False, False)], 90: [(1, False, False)], 91: [(1, False, False)], 92: [(1, False, False)]}
Human 1 is detected in 32

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/21/tracks/21.txt')
print(came, left, violations)

(0, 1)
cntry_min = 288, cntry_max = 372
cntry_start = 290, cntry_end = 349
Human 1 moved 24% of the frame (vertically)
(0, 5)
cntry_min = 379, cntry_max = 396
cntry_start = 390, cntry_end = 394
Human 5 moved 5% of the frame (vertically)
(0, 10)
cntry_min = 380, cntry_max = 398
cntry_start = 383, cntry_end = 397
Human 10 moved 5% of the frame (vertically)
(0, 20)
cntry_min = 291, cntry_max = 417
cntry_start = 417, cntry_end = 291
Human 20 moved 37% of the frame (vertically)
Human 20 left.
[] [20]
0 people came in
1 people left
{44: [(1, False, False)], 45: [(1, False, False)], 46: [(1, False, False)], 47: [(1, False, False)], 48: [(1, False, False)], 49: [(1, False, False)], 50: [(1, False, False)], 51: [(1, False, False)], 52: [(1, False, False)], 53: [(1, False, False)], 54: [(1, False, False)], 55: [(1, False, False)], 56: [(1, False, False)], 57: [(1, False, False)], 58: [(1, False, False)], 59: [(1, False, False)], 60: [(1, False, False)], 61: [(1, False, False)], 62: [(1, False, F

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/22/tracks/22.txt')
print(came, left, violations)

(0, 1)
cntry_min = 285, cntry_max = 396
cntry_start = 396, cntry_end = 296
Human 1 moved 32% of the frame (vertically)
Human 1 left.
(0, 4)
cntry_min = 277, cntry_max = 415
cntry_start = 415, cntry_end = 280
Human 4 moved 40% of the frame (vertically)
Human 4 left.
(0, 10)
cntry_min = 286, cntry_max = 404
cntry_start = 402, cntry_end = 288
Human 10 moved 34% of the frame (vertically)
Human 10 left.
[] [1, 4, 10]
0 people came in
3 people left
{51: [(1, False, False)], 52: [(1, False, False)], 53: [(1, False, False)], 54: [(1, False, False)], 55: [(1, False, False)], 56: [(1, False, False)], 57: [(1, False, False)], 58: [(1, False, False)], 59: [(1, False, False)], 60: [(1, False, False)], 61: [(1, False, False)], 62: [(1, False, False)], 63: [(1, False, False)], 64: [(1, False, False)], 65: [(1, False, False)], 66: [(1, False, False)], 67: [(1, False, False)], 68: [(1, False, False)], 69: [(1, False, False)], 70: [(1, False, False)], 71: [(1, False, False)], 72: [(1, False, False)], 73

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/23/tracks/23.txt')
print(came, left, violations)

(0, 1)
cntry_min = 291, cntry_max = 411
cntry_start = 411, cntry_end = 295
Human 1 moved 37% of the frame (vertically)
Human 1 left.
[] [1]
0 people came in
1 people left
{57: [(1, False, False)], 58: [(1, False, False)], 59: [(1, False, False)], 60: [(1, False, False)], 61: [(1, False, False)], 62: [(1, False, False)], 63: [(1, False, False)], 64: [(1, False, False)], 65: [(1, False, False)], 66: [(1, False, False)], 67: [(1, False, False)], 68: [(1, False, False)], 69: [(1, False, False)], 70: [(1, False, False)], 71: [(1, False, False)], 72: [(1, False, False)], 73: [(1, False, False)], 74: [(1, False, False)], 75: [(1, False, False)], 76: [(1, False, False)], 77: [(1, False, False)], 78: [(1, False, False)], 79: [(1, False, False)], 80: [(1, False, False)], 81: [(1, False, False)], 82: [(1, False, False)], 83: [(1, False, False)], 84: [(1, False, False)], 85: [(1, False, False)], 86: [(1, False, False)], 87: [(1, False, False)], 88: [(1, False, False)], 89: [(1, False, False)], 90:

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/24/tracks/24.txt')
print(came, left, violations)

(0, 1)
cntry_min = 292, cntry_max = 445
cntry_start = 292, cntry_end = 444
Human 1 moved 47% of the frame (vertically)
Human 1 came in.
[1] []
1 people came in
0 people left
{38: [(1, False, True)], 39: [(1, False, True)], 40: [(1, False, True)], 41: [(1, False, True)], 42: [(1, True, True)], 43: [(1, True, True)], 44: [(1, True, True)], 45: [(1, True, True)], 46: [(1, True, True)], 47: [(1, True, True)], 48: [(1, True, True)], 49: [(1, True, True)], 50: [(1, True, True)], 51: [(1, True, True)], 52: [(1, True, True)], 53: [(1, True, True)], 54: [(1, True, True)], 55: [(1, True, True)], 56: [(1, True, True)], 57: [(1, True, True)], 58: [(1, True, True)], 59: [(1, True, True)], 60: [(1, True, True)], 61: [(1, True, True)], 62: [(1, True, True)], 63: [(1, True, True)], 64: [(1, True, True)], 65: [(1, True, True)], 66: [(1, True, True)], 67: [(1, True, True)], 68: [(1, True, True)], 69: [(1, True, True)], 70: [(1, True, True)], 71: [(1, True, True)], 72: [(1, True, True)], 73: [(1, True, T

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/25/tracks/25.txt')
print(came, left, violations)

(0, 1)
cntry_min = 263, cntry_max = 498
cntry_start = 263, cntry_end = 498
Human 1 moved 39% of the frame (vertically)
Human 1 came in.
(0, 2)
cntry_min = 239, cntry_max = 476
cntry_start = 258, cntry_end = 472
Human 2 moved 39% of the frame (vertically)
Human 2 came in.
[1, 2] []
2 people came in
0 people left
{21: [(1, False, False)], 22: [(1, False, False)], 23: [(1, False, False)], 24: [(1, False, False)], 25: [(1, False, False)], 26: [(1, False, False)], 27: [(1, False, False)], 28: [(1, False, False)], 29: [(1, False, False)], 30: [(1, False, False)], 31: [(1, False, False)], 32: [(1, False, False)], 33: [(1, False, False)], 34: [(1, False, False)], 35: [(1, False, False)], 36: [(1, False, False)], 37: [(1, False, False)], 38: [(1, False, False)], 39: [(1, False, False)], 40: [(1, False, False)], 41: [(1, False, False)], 42: [(1, False, False)], 43: [(1, False, False)], 44: [(1, False, False)], 45: [(1, False, False)], 46: [(1, False, False)], 47: [(1, False, False)], 48: [(1, Fa

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/26/tracks/26.txt')
print(came, left, violations)

(0, 1)
cntry_min = 233, cntry_max = 454
cntry_start = 454, cntry_end = 233
Human 1 moved 36% of the frame (vertically)
Human 1 left.
[] [1]
0 people came in
1 people left
{75: [(1, False, False)], 76: [(1, False, False)], 77: [(1, False, False)], 78: [(1, False, False)], 79: [(1, False, False)], 80: [(1, False, False)], 81: [(1, False, False)], 82: [(1, False, False)], 83: [(1, False, False)], 84: [(1, False, False)], 85: [(1, False, False)], 86: [(1, False, False)], 87: [(1, False, False)], 88: [(1, False, False)], 89: [(1, False, False)], 90: [(1, False, False)], 91: [(1, False, False)], 92: [(1, False, False)], 93: [(1, False, False)], 94: [(1, False, False)], 95: [(1, False, False)], 96: [(1, False, False)], 97: [(1, False, False)], 98: [(1, False, False)], 99: [(1, False, False)], 100: [(1, False, False)], 101: [(1, False, False)], 102: [(1, False, False)], 103: [(1, False, False)], 104: [(1, False, False)], 105: [(1, False, False)], 106: [(1, False, False)], 107: [(1, False, Fals

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/27/tracks/27.txt')
print(came, left, violations)

(0, 1)
cntry_min = 284, cntry_max = 517
cntry_start = 284, cntry_end = 517
Human 1 moved 39% of the frame (vertically)
Human 1 came in.
(0, 5)
cntry_min = 245, cntry_max = 454
cntry_start = 454, cntry_end = 256
Human 5 moved 35% of the frame (vertically)
Human 5 left.
[1] [5]
1 people came in
1 people left
{29: [(1, False, False)], 30: [(1, False, False)], 31: [(1, False, False)], 32: [(1, False, False)], 33: [(1, False, False)], 34: [(1, False, False)], 35: [(1, False, False)], 36: [(1, False, False)], 37: [(1, False, False)], 38: [(1, False, False)], 39: [(1, False, False)], 40: [(1, False, False)], 41: [(1, False, False)], 42: [(1, False, False)], 43: [(1, False, False)], 44: [(1, False, False)], 45: [(1, False, False)], 46: [(1, False, False)], 47: [(1, False, False)], 48: [(1, False, False)], 49: [(1, False, False)], 50: [(1, False, False)], 51: [(1, False, False)], 52: [(1, False, False)], 53: [(1, False, False)], 54: [(1, False, False)], 55: [(1, False, False)], 56: [(1, False, 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/28/tracks/28.txt')
print(came, left, violations)

(0, 1)
cntry_min = 284, cntry_max = 517
cntry_start = 284, cntry_end = 517
Human 1 moved 39% of the frame (vertically)
Human 1 came in.
(0, 5)
cntry_min = 245, cntry_max = 454
cntry_start = 454, cntry_end = 256
Human 5 moved 35% of the frame (vertically)
Human 5 left.
[1] [5]
1 people came in
1 people left
{29: [(1, False, False)], 30: [(1, False, False)], 31: [(1, False, False)], 32: [(1, False, False)], 33: [(1, False, False)], 34: [(1, False, False)], 35: [(1, False, False)], 36: [(1, False, False)], 37: [(1, False, False)], 38: [(1, False, False)], 39: [(1, False, False)], 40: [(1, False, False)], 41: [(1, False, False)], 42: [(1, False, False)], 43: [(1, False, False)], 44: [(1, False, False)], 45: [(1, False, False)], 46: [(1, False, False)], 47: [(1, False, False)], 48: [(1, False, False)], 49: [(1, False, False)], 50: [(1, False, False)], 51: [(1, False, False)], 52: [(1, False, False)], 53: [(1, False, False)], 54: [(1, False, False)], 55: [(1, False, False)], 56: [(1, False, 

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/29/tracks/29.txt')
print(came, left, violations)

(0, 2)
cntry_min = 261, cntry_max = 569
cntry_start = 474, cntry_end = 569
Human 2 moved 54% of the frame (vertically)
Human 2 left and came back.
[2] [2]
1 people came in
1 people left
{33: [(2, False, False)], 34: [(2, False, False)], 35: [(2, False, False)], 36: [(2, False, False)], 37: [(2, False, False)], 38: [(2, False, False)], 39: [(2, False, False)], 40: [(2, False, False)], 41: [(2, False, False)], 42: [(2, False, False)], 43: [(2, False, False)], 44: [(2, False, False)], 45: [(2, False, False)], 46: [(2, False, False)], 47: [(2, False, False)], 48: [(2, False, False)], 49: [(2, False, False)], 50: [(2, False, False)], 51: [(2, False, False)], 52: [(2, False, False)], 53: [(2, False, False)], 54: [(2, False, False)], 55: [(2, False, False)], 56: [(2, False, False)], 57: [(2, False, False)], 58: [(2, False, False)], 59: [(2, False, False)], 60: [(2, False, False)], 66: [(2, False, False)], 67: [(2, False, False)], 68: [(2, False, False)], 69: [(2, False, False)], 70: [(2, Fals

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/30/tracks/30.txt')
print(came, left, violations)

(0, 1)
cntry_min = 262, cntry_max = 486
cntry_start = 272, cntry_end = 486
Human 1 moved 38% of the frame (vertically)
Human 1 came in.
(0, 4)
cntry_min = 241, cntry_max = 304
cntry_start = 278, cntry_end = 259
Human 4 moved 10% of the frame (vertically)
(0, 6)
cntry_min = 518, cntry_max = 520
cntry_start = 520, cntry_end = 518
Human 6 moved 0% of the frame (vertically)
(0, 7)
cntry_min = 541, cntry_max = 542
cntry_start = 541, cntry_end = 542
Human 7 moved 0% of the frame (vertically)
(0, 8)
cntry_min = 496, cntry_max = 517
cntry_start = 501, cntry_end = 517
Human 8 moved 3% of the frame (vertically)
(0, 9)
cntry_min = 463, cntry_max = 520
cntry_start = 505, cntry_end = 520
Human 9 moved 9% of the frame (vertically)
(0, 11)
cntry_min = 406, cntry_max = 454
cntry_start = 454, cntry_end = 406
Human 11 moved 8% of the frame (vertically)
(0, 14)
cntry_min = 296, cntry_max = 468
cntry_start = 468, cntry_end = 296
Human 14 moved 29% of the frame (vertically)
Human 14 left.
(0, 17)
cntry_min

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/31/tracks/31.txt')
print(came, left, violations)

(0, 2)
cntry_min = 276, cntry_max = 380
cntry_start = 380, cntry_end = 281
Human 2 moved 30% of the frame (vertically)
Human 2 left.
(0, 6)
cntry_min = 266, cntry_max = 412
cntry_start = 412, cntry_end = 266
Human 6 moved 42% of the frame (vertically)
Human 6 left.
[] [2, 6]
0 people came in
2 people left
{18: [(2, True, True)], 19: [(2, True, True)], 20: [(2, True, True)], 21: [(2, True, True)], 22: [(2, True, True)], 23: [(2, True, True)], 24: [(2, True, True)], 25: [(2, True, True)], 26: [(2, True, True)], 27: [(2, True, True)], 28: [(2, True, True)], 29: [(2, True, True)], 30: [(2, True, True)], 31: [(2, True, True)], 32: [(2, True, True)], 33: [(2, True, True)], 34: [(2, True, True)], 35: [(2, True, True)], 36: [(2, True, True)], 37: [(2, True, True)], 38: [(2, True, True)], 39: [(2, True, True)], 40: [(2, True, True)], 41: [(2, True, True)], 42: [(2, True, True)], 43: [(2, True, True)], 44: [(2, True, True)], 45: [(2, True, True)], 46: [(2, True, True)], 47: [(2, True, True)], 48

Файлы 32.txt, 33.txt, 34.txt, 35.txt, 37.txt, 41.txt, 43.txt не существуют, так как на видео никто не проходит и ничего не детектится.

Необходимо либо дописать перехват эксепшена, чибо, что возможно лучше, попробовать вписать весь код в файл track.py из репозитория, тем более что он и так уже модифицирован, дальнейшая интеграция также уберет необходимость отдельного запуска трекера и отдельного запуска функции постобработчика.

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/36/tracks/36.txt')
print(came, left, violations)

(0, 1)
cntry_min = 312, cntry_max = 324
cntry_start = 324, cntry_end = 313
Human 1 moved 4% of the frame (vertically)
[] []
0 people came in
0 people left
{3: [(1, False, False)], 4: [(1, False, False)], 5: [(1, False, False)], 6: [(1, False, False)], 7: [(1, False, False)], 8: [(1, False, False)], 9: [(1, False, False)], 10: [(1, False, False)], 11: [(1, False, False)], 12: [(1, False, False)], 13: [(1, False, False)], 14: [(1, False, False)], 15: [(1, False, False)], 16: [(1, False, False)], 17: [(1, False, False)], 18: [(1, False, False)], 19: [(1, False, False)], 20: [(1, False, False)], 21: [(1, False, False)], 22: [(1, False, False)], 23: [(1, False, False)], 24: [(1, False, False)], 25: [(1, False, False)], 26: [(1, False, False)], 27: [(1, False, False)], 28: [(1, False, False)], 29: [(1, False, False)], 30: [(1, False, False)], 31: [(1, False, False)], 32: [(1, False, False)], 33: [(1, False, False)], 34: [(1, False, False)], 35: [(1, False, False)], 36: [(1, False, False)], 3

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/38/tracks/38.txt')
print(came, left, violations)

(0, 1)
cntry_min = 295, cntry_max = 390
cntry_start = 299, cntry_end = 390
Human 1 moved 26% of the frame (vertically)
Human 1 came in.
(0, 2)
cntry_min = 289, cntry_max = 372
cntry_start = 291, cntry_end = 349
Human 2 moved 23% of the frame (vertically)
(0, 4)
cntry_min = 379, cntry_max = 396
cntry_start = 390, cntry_end = 394
Human 4 moved 4% of the frame (vertically)
(0, 7)
cntry_min = 379, cntry_max = 399
cntry_start = 394, cntry_end = 397
Human 7 moved 5% of the frame (vertically)
(0, 16)
cntry_min = 287, cntry_max = 419
cntry_start = 419, cntry_end = 287
Human 16 moved 37% of the frame (vertically)
Human 16 left.
(0, 18)
cntry_min = 286, cntry_max = 396
cntry_start = 396, cntry_end = 296
Human 18 moved 31% of the frame (vertically)
Human 18 left.
(0, 21)
cntry_min = 277, cntry_max = 415
cntry_start = 415, cntry_end = 281
Human 21 moved 39% of the frame (vertically)
Human 21 left.
(0, 26)
cntry_min = 285, cntry_max = 404
cntry_start = 402, cntry_end = 285
Human 26 moved 33% of the

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/39/tracks/39.txt')
print(came, left, violations)

(0, 3)
cntry_min = 273, cntry_max = 365
cntry_start = 365, cntry_end = 273
Human 3 moved 27% of the frame (vertically)
Human 3 left.
[] [3]
0 people came in
1 people left
{68: [], 69: [], 70: [(3, True, True)], 71: [(3, True, True)], 72: [(3, True, True)], 73: [(3, True, True)], 74: [(3, True, True)], 75: [(3, True, True)], 76: [(3, True, True)], 77: [(3, True, True)], 78: [(3, True, True)], 79: [(3, True, True)], 80: [(3, True, True)], 81: [(3, True, True)], 82: [(3, True, True)], 83: [(3, True, True)], 84: [(3, True, True)], 85: [(3, True, True)], 86: [(3, True, True)], 87: [(3, True, True)], 88: [(3, True, True)], 89: [(3, True, True)], 90: [(3, True, True)], 91: [(3, True, True)], 92: [(3, True, True)], 93: [(3, True, True)], 94: [(3, True, True)], 95: [(3, True, True)], 96: [(3, True, True)], 97: [(3, True, True)], 98: [(3, True, True)], 99: [(3, True, True)], 100: [(3, True, True)], 101: [(3, True, True)], 102: [(3, True, True)], 103: [(3, True, True)], 104: [(3, True, True)], 10

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/40/tracks/40.txt')
print(came, left, violations)

(0, 1)
cntry_min = 288, cntry_max = 421
cntry_start = 411, cntry_end = 289
Human 1 moved 41% of the frame (vertically)
Human 1 left.
[] [1]
0 people came in
1 people left
{165: [(1, False, False)], 166: [(1, False, False)], 167: [(1, False, False)], 168: [(1, False, False)], 169: [(1, False, False)], 170: [(1, False, False)], 171: [(1, False, False)], 172: [(1, False, False)], 173: [(1, False, False)], 174: [(1, False, False)], 175: [(1, False, False)], 176: [(1, False, False)], 177: [(1, False, False)], 215: [(1, False, False)], 216: [(1, False, False)], 217: [(1, False, False)], 218: [(1, False, False)], 219: [(1, False, False)], 220: [(1, False, False)], 221: [(1, False, False)], 222: [(1, False, False)], 223: [(1, False, False)], 224: [(1, False, False)], 225: [(1, False, False)], 226: [(1, False, False)], 227: [(1, False, False)], 228: [(1, False, False)], 229: [(1, False, False)], 230: [(1, False, False)], 231: [(1, False, False)], 232: [(1, False, False)], 233: [(1, False, False

In [ ]:
came, left, violations = get_results('/content/drive/MyDrive/Hardhats_vests/tracker_results/42/tracks/42.txt')
print(came, left, violations)

(0, 1)
cntry_min = 306, cntry_max = 401
cntry_start = 306, cntry_end = 401
Human 1 moved 29% of the frame (vertically)
Human 1 came in.
(0, 2)
cntry_min = 294, cntry_max = 435
cntry_start = 295, cntry_end = 435
Human 2 moved 44% of the frame (vertically)
Human 2 came in.
(0, 4)
cntry_min = 295, cntry_max = 317
cntry_start = 298, cntry_end = 308
Human 4 moved 6% of the frame (vertically)
[1, 2] []
2 people came in
0 people left
{3: [(1, False, False)], 4: [(1, False, False)], 5: [(1, False, False)], 6: [(1, False, False)], 7: [(1, False, False)], 8: [(1, False, False)], 9: [(1, False, False)], 10: [(1, False, False)], 11: [(1, False, False)], 12: [(1, False, False)], 13: [(1, False, False)], 14: [(1, False, False)], 15: [(1, False, False)], 16: [(1, False, False)], 17: [(1, False, False)], 18: [(1, False, False)], 19: [(1, False, False)], 20: [(1, False, False)], 21: [(1, False, False)], 22: [(1, False, False)], 23: [(1, False, False)], 24: [(1, False, False)], 25: [(1, False, False)], 

| Видео           | Описание        | Детекция        | Комментарий     |      
| --------------- | --------------- | --------------- | --------------- |
| 1.mp4           | ЧКЖ+ ЧКЖ+ ЧКЖ+  | 3+ 0-           | ОК              |
| 2.mp4           | ЧЖ+ ЧКЖ+ ЧКЖ+   | 3+ 0-           | OK (нарушение замечено верно)        |
| 3.mp4           | Ч ЧКЖ           | 0+ 0-           | OK              |
| 4.mp4           |ЧКЖ+ Ч+ ЧКЖ+ ЧКЖ+| 4+ 0-           | OK (нарушение замечено верно) |
| 5.mp4           | ЧКЖ ЧЖ          | 1+ 0-           | ошибка счета (охранник наклоняется через турникет, центр рамки пересекает турникет)              |
| 6.mp4           |Ч- Ч- Ч- Ч- ЧКЖ+ | 1+ 3-          | ошибка счета (один из выходящих долго загорожен другим)|
|                 |                 |                 | ошибка детекции нарушения (жилет мало детектится сетью (расстегнут))                |
| 7.mp4           | ЧЖ- Ч- ЧЖ- Ч-   | 0+ 4-           | OK (нарушение замечено верно)|
| 8.mp4           | Ч- Ч- Ч- Ч- Ч-  | 0+ 4-           | ошибка счета (второй выходящий детектится не все время, поэтому перемещается < 25% кадра (пороговое значение), в правом нижнем углу рядом с дверью охранника часто косяки детекции) |
|                 |                 |                 | нарушения замечены верно |
| 9.mp4           | ЧЖ- ЧКЖ ЧКЖ+    | 1+ 1-           | OK (нарушение замечено верно) |
| 10.mp4          | ЧЖ- ЧЖ-         | 0+ 1-           | ошибка счета (совершенно непонятная внезапная смена id второго выходящего человека), нарушение замечено верно |
| 11.mp4          | Ч- Ч- ЧЖ-       | 0+ 3-           | ОК (нарушения замечены верно) |
| 12.mp4          | Ч- Ч- Ч- ЧКЖ-   | 0+ 4-           | ОК (нарушения замечены верно) |
| 13.mp4          | ЧКЖ- ЧЖ- Ч- Ч- ЧЖ+ ЧЖ+ | 2+ 4-    | Счет верен несмотря на неверно присвоенный id 16, но из-за этого неверно замечен тип нарушения. У 3го выходящего оранж. куртк., но с ним все прекрасно!)) |
| 14.mp4          | ЧКЖ- ЧКЖ- ЧКЖ-  | 0+ 3-           | Счет верен несмотря на полный трэш с рамками |
|                 |                 |                 | ошибка детекции нарушения (каска полностью за бб человека, неверная детекция бб человека) |
| 15.mp4          | Ч+ Ч+           | 2+ 0-           | ОК (нарушения замечены верно) |
| 16.mp4          | ЧЖ- ЧЖ- ЧКЖ- Ч- | 0+ 4-           | ОК (нарушения замечены верно) |
| 17.mp4          |Ч- ЧКЖ- Ч- Ч- ЧЖ- ЧЖ-| 0+ 5-       | ошибка счета (совершенно непонятная внезапная смена id третьего выходящего человека)                |
|                 |                 |                 | У второго выходящего синяя каска не детектится, остальные нарушения детектятся верно. |
| 18.mp4          |ЧКЖ+ ЧКЖ+ ЧК+ ЧК+| 4+ 0-           | ОК (нарушения замечены верно)|
| 19.mp4          | ЧЖ+             | 1+ 0-           | ОК (нарушения замечены верно)|
| 20.mp4          | Ч+              | 1+ 0-           | ОК (нарушения замечены верно)|
| 21.mp4          | Ч+ Ч-           | 0+ 1-           | ошибка счета (смена id человека, который вошел и вышел, вход не учтен, т.к. центр бб сдвинулся на 24% (пороговое значение 25%)|
|                 |                 |                 |нарушение замечено верно|
| 22.mp4          | Ч- ЧКЖ- Ч-      | 0+ 3-           | ОК (нарушения замечены верно)|
| 23.mp4          | Ч-              | 0+ 1-           | ОК (нарушения замечены верно)|
| 24.mp4          | ЧКЖ+            | 1+ 0-           | ОК              |
| 25.mp4          | Ч+ Ч+           | 2+ 0-           | ОК (нарушения замечены верно)|
| 26.mp4          | Ч-              | 0+ 1-           | ОК (нарушения замечены верно) |
| 27.mp4          | Ч+ ЧКЖ-         | 1+ 1-           | ошибочная детекция нарушения (каска детектится на 34 кадрах из 60, меньше порога в 70%) |
| 28.mp4          | дубликат 27.mp4 |                 |                 |
| 29.mp4          | Ч- Ч+           | 1+ 1-           | ОК несмотря на неверное присвоение id |
| 30.mp4          | Ч+ ЧЖ- ЧЖ- ЧЖ-  | 1+ 2-           | ошибка счета (у первого выходящего нет бб человека!!!)|
|                 |                 |                 |нарушения замечены верно|
| 31.mp4          | ЧКЖ- ЧКЖ-       | 0+ 2-           | ошибка детекции нарушения (жилет детектится в 46 кадрах из 82, меньше порога в 70%|
| 32.mp4          |никто не проходит|                 |                 |
| 33.mp4          |никто не проходит|                 |                 |
| 34.mp4          |никто не проходит|                 |                 |
| 35.mp4          |никто не проходит|                 |                 |
| 36.mp4          | Ч               | 0+ 0-           | ОК              |
| 37.mp4          |никто не проходит|                 |                 |
| 38.mp4          |Ч+ Ч+ Ч- Ч- ЧКЖ- Ч- Ч- ЧКЖ+|из 8+ 7- посчитано 6+ 6-  |второй человек не засчитан, т.к. переместился на 23% кадра, а пороговое значение 25%|
|                 |Ч+ Ч+ Ч- Ч+ Ч+ ЧКЖ- ЧКЖ+|                 |ошибка детекции нарушения из-зи неверно присвоеного id|
|                 |                 |                 |ошибка подсчета, не учтены последние 2 человека, из-зи неверно присвоенного id|
| 39.mp4          | ЧКЖ-            | 0+ 1-           | ОК              |
| 40.mp4          | Ч-              | 0+ 1-           | ОК (нарушения замечены верно)|
| 41.mp4          |никто не проходит|                 |                 |
| 42.mp4          | Ч+ Ч+           | 2+ 0-           |ОК (нарушения замечены верно)|
| 43.mp4          |никто не проходит|                 |                 |
|                 |                 |                 |                 |


**Вывод:**

Всего на видео присутствуют 107 человек, из них правильно учтены (входит, выходит, не пересекает турникет) 97, что примерно равняется точности в 91%

Ошибок детекции нарушений 6 из 107 человек, точность 94%

Возможно показатели могут быть улучшены сменой пороговых значений, но для начала следует улучшить/пополнить датасет и/или его разметку.
Возможно смена трекера (со strongsort на другой) поможет избавиться от внезапной смены id.